In [35]:
# Import Libraries and Dependencies
import pandas as pd


### Read in files

In [55]:
# Read in data and index by CustomerID, STUDENT NOTE using order_id as index since there is not customer id
# File paths
jan_2019_sales = pd.read_csv("Resources/Jan2019_sales.csv", index_col="order_ID")
feb_2019_sales = pd.read_csv("Resources/Feb2019_sales.csv", index_col="order_ID")
mar_2019_sales = pd.read_csv("Resources/March2019_sales.csv", index_col="order_ID")



In [56]:
# Show sample of January sales data.
jan_2019_sales.head()


,product,quantity_ordered,price,order_date,state,city,zip_code
order_ID,,,,,,,
141234,iPhone,1,700.00,01/22/19 21:25,FL,Orlando,32807
141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,FL,Miami,33137
141236,Wired Headphones,2,11.99,01/17/19 13:33,GA,Atlanta,30318
141237,27in FHD Monitor,1,149.99,01/05/19 20:33,CA,Los Angeles,90014
141238,Wired Headphones,1,11.99,01/25/19 11:59,CA,San Francisco,94016


In [41]:
# Show sample of February sales data.
feb_2019_sales.head()



,order_ID,product,quantity_ordered,price,order_date,state,city,zip_code
0,150502,iPhone,1,700.00,02/18/19 01:35,PA,Philadelphia,19132
1,150503,AA Batteries (4-pack),1,3.84,02/13/19 07:24,CA,San Diego,91911
2,150504,27in 4K Gaming Monitor,1,389.99,02/18/19 09:46,CA,San Diego,91911
3,150505,Lightning Charging Cable,1,14.95,02/02/19 16:47,FL,Orlando,32822
4,150506,AA Batteries (4-pack),2,3.84,02/28/19 20:32,WA,Seattle,98115


In [42]:
# Show sample of March sales data.
mar_2019_sales.head()


,order_ID,product,quantity_ordered,price,order_date,state,city,zip_code
0,162009,iPhone,1,700.00,03/28/19 20:59,CA,Los Angeles,90008
1,162009,Lightning Charging Cable,1,14.95,03/28/19 20:59,TX,Austin,78729
2,162009,Wired Headphones,2,11.99,03/28/19 20:59,OR,Portland,97209
3,162010,Bose SoundSport Headphones,1,99.99,03/17/19 05:39,FL,Miami,33125
4,162011,34in Ultrawide Monitor,1,379.99,03/10/19 00:01,CA,San Diego,92029


### Check the data types of each DataFrame

In [43]:
# Jan sales datatypes
jan_2019_sales.dtypes

order_ID              int64
product              object
quantity_ordered      int64
price               float64
order_date           object
state                object
city                 object
zip_code              int64
dtype: object

In [44]:
# Feb sales datatypes
feb_2019_sales.dtypes

order_ID              int64
product              object
quantity_ordered      int64
price               float64
order_date           object
state                object
city                 object
zip_code              int64
dtype: object

In [46]:
# March sales datatypes
mar_2019_sales.dtypes


order_ID              int64
product              object
quantity_ordered      int64
price               float64
order_date           object
state                object
city                 object
zip_code              int64
dtype: object

### Combine the sales data by rows.

In [47]:
# Combine the January, February, and March sales DataFrames on the rows using an inner join, and reset the index
combined_sales = pd.concat([jan_2019_sales, feb_2019_sales, mar_2019_sales], axis="rows", join="inner")
combined_sales.head()


,order_ID,product,quantity_ordered,price,order_date,state,city,zip_code
0,141234,iPhone,1,700.00,01/22/19 21:25,FL,Orlando,32807
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,FL,Miami,33137
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,GA,Atlanta,30318
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,CA,Los Angeles,90014
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,CA,San Francisco,94016


In [48]:
# Check if any values are null.
combined_sales.isnull().sum()


order_ID            0
product             0
quantity_ordered    0
price               0
order_date          0
state               0
city                0
zip_code            0
dtype: int64

In [49]:
# Check the data type of each column
combined_sales.dtypes


order_ID              int64
product              object
quantity_ordered      int64
price               float64
order_date           object
state                object
city                 object
zip_code              int64
dtype: object

In [57]:
# Convert the "order_date" column to a datetime datatype.
combined_sales["order_date"] = pd.to_datetime(combined_sales["order_date"])


In [52]:
# Confirm that the data type has been changed.
combined_sales.dtypes


order_ID                     int64
product                     object
quantity_ordered             int64
price                      float64
order_date          datetime64[ns]
state                       object
city                        object
zip_code                     int64
dtype: object

### 1. Which top five zip codes in which cities and states had the greatest number of products ordered?

### Using `groupby`

In [61]:
# Show the average number items ordered for each state, city, and zip code.
# Rename the sum column to reflect the data in the column.
# Sort in descending order.
# Group by state, city, and zip code, then calculate the average number of items
average_items = combined_sales.groupby(['state', 'city', 'zip_code'])['quantity_ordered'].mean()

# Rename the resulting series to reflect the data
average_items.rename('average_items_ordered', inplace=True)

# Display the top 5 results
top_5_average_items = average_items.nlargest(5)
top_5_average_items

state  city           zip_code
IL     Chicago        60629       1.166881
CA     San Francisco  94102       1.152958
FL     Miami          33125       1.151847
MI     Detroit        48215       1.150298
CA     San Francisco  94016       1.149860
Name: average_items_ordered, dtype: float64

### Using `pivot_table`

In [63]:
# Show the total number of items ordered for each state, city, and zip code.
pivot_table = combined_sales.pivot_table(
    values='quantity_ordered',  # The data to aggregate
    index=['state', 'city', 'zip_code'],  # Columns to group by
    aggfunc='mean'  # Aggregation function
)


# Rename the "sum" column to "Avg_Number_Products_Ordered"
pivot_table.columns = ['average_items_ordered']

# Show the top 5 results.
pivot_table


average_items_ordered
state city          zip_code                       
CA    Los Angeles   90001                  1.131436
                    90008                  1.114168
                    90014                  1.111878
      San Diego     91911                  1.112532
                    92029                  1.142670
                    92091                  1.109439
      San Francisco 94016                  1.149860
                    94102                  1.152958
                    94118                  1.104336
FL    Miami         33125                  1.151847
                    33137                  1.133148
                    33186                  1.113833
      Orlando       32801                  1.135642
                    32807                  1.117730
                    32822                  1.119718
GA    Atlanta       30303                  1.096317
                    30318                  1.133240
                    30339                  1.123611
IL    Chicago       60607                  1.137042
                    60618                  1.120464
                    60629                  1.166881
MI    Detroit       48127                  1.108333
                    48207                  1.119403
                    48215                  1.150298
NY    Buffalo       14201                  1.111111
                    14205                  1.127119
                    14211                  1.119488
      New York City 10001                  1.110256
                    10013                  1.106105
                    10118                  1.094840
OH    Columbus      43201                  1.130548
                    43212                  1.105783
                    43224                  1.108853
OR    Portland      97201                  1.127223
                    97209                  1.137834
                    97223                  1.129604
PA    Philadelphia  19102                  1.125180
                    19116                  1.125758
                    19132                  1.109459
TX    Austin        78701                  1.148850
                    78729                  1.100282
                    78745                  1.126126
      Dallas        75201                  1.111425
                    75209                  1.109788
                    75215                  1.113032
      Houston       77002                  1.142462
                    77019                  1.080780
                    77055                  1.120160
WA    Seattle       98101                  1.120166
                    98115                  1.137466
                    98122                  1.113960

### 2. Which top five zip codes in which cities and states generated the most sales?

### Using `groupby`

In [65]:
# Show the total price for the items ordered for each state, city, and zip code.
# Rename the sum column to reflect the data in the column.
total_price_groupby = combined_sales.groupby(['state', 'city', 'zip_code'])['price'].sum()

# Rename the resulting series to reflect the data
total_price_groupby.rename('total_price_sum', inplace=True)

# Display the top 5 results
top_5_total_price_groupby = total_price_groupby.nlargest(5)
top_5_total_price_groupby


state  city           zip_code
NY     Buffalo        14201       155976.70
IL     Chicago        60629       150667.34
CA     San Diego      92029       150305.41
       San Francisco  94102       146115.98
NY     New York City  10013       145934.27
Name: total_price_sum, dtype: float64

### Using `pivot_table`

In [66]:
# Show the total price for all of the items ordered for each state, city, and zip code.
# Create a pivot table for the total price
total_price_pivot = combined_sales.pivot_table(
    values='price',  # Assuming 'price' represents the total price of items ordered
    index=['state', 'city', 'zip_code'],  # Columns to group by
    aggfunc='sum'  # Sum up the total prices
)

# Rename the aggregated column to "Avg_Number_Products_Ordered"
total_price_pivot.columns = ['Avg_Number_Products_Ordered']

# Sort the pivot table in descending order and display the top 5 results
top_5_total_price = total_price_pivot.sort_values(by='Avg_Number_Products_Ordered', ascending=False).head(5)
top_5_total_price


Avg_Number_Products_Ordered
state city          zip_code                             
NY    Buffalo       14201                       155976.70
IL    Chicago       60629                       150667.34
CA    San Diego     92029                       150305.41
      San Francisco 94102                       146115.98
NY    New York City 10013                       145934.27

### 3. Which top five zip codes in which cities and states had the greatest average price for the products ordered?

### Using `groupby`

In [67]:
# Show the average price of items ordered for each state, city, and zip code rounded to two decimal places.
# Rename the mean column to reflect the data in the column.
average_price_groupby = combined_sales.groupby(['state', 'city', 'zip_code'])['price'].mean()

# Round the results to two decimal places
average_price_groupby = average_price_groupby.round(2)

# Rename the resulting series to reflect the data
average_price_groupby.rename('average_item_price', inplace=True)

# Display the top 5 results
top_5_average_price_groupby = average_price_groupby.nlargest(5)
top_5_average_price_groupby

state  city           zip_code
NY     New York City  10013       212.11
CA     San Francisco  94102       210.85
TX     Austin         78745       205.63
NY     Buffalo        14201       203.89
CA     San Diego      92029       196.73
Name: average_item_price, dtype: float64

### Using `pivot_table`

In [68]:
# Show the average price per product ordered for each state, city, and zip code rounded to two decimal places.
average_price_pivot = combined_sales.pivot_table(
    values='price',  # Assuming 'price' represents the price per product
    index=['state', 'city', 'zip_code'],  # Columns to group by
    aggfunc='mean'  # Calculate the average
)

# Round the results to two decimal places
average_price_pivot = average_price_pivot.round(2)

# Rename the aggregated column to "Avg_Price"
average_price_pivot.columns = ['Avg_Price']

# Sort the pivot table in descending order and display the top 5 results
top_5_average_price = average_price_pivot.sort_values(by='Avg_Price', ascending=False).head(5)
top_5_average_price


,,,Avg_Price
state,city,zip_code,
NY,New York City,10013,212.11
CA,San Francisco,94102,210.85
TX,Austin,78745,205.63
NY,Buffalo,14201,203.89
CA,San Diego,92029,196.73


### 4. Which top five zip codes in which cities and states had the most iPhone sales?

In [70]:
# Filter the sales data to get the iPhone sales data.
iphone_sales = combined_sales.loc[combined_sales["product"] == "iPhone"]
iphone_sales.head()


,order_ID,product,quantity_ordered,price,order_date,state,city,zip_code
0,141234,iPhone,1,700.0,2019-01-22 21:25:00,FL,Orlando,32807
104,141336,iPhone,1,700.0,2019-01-09 18:23:00,IL,Chicago,60629
164,141394,iPhone,1,700.0,2019-01-06 16:54:00,TX,Dallas,75201
207,141437,iPhone,1,700.0,2019-01-10 15:40:00,FL,Miami,33125
228,141457,iPhone,1,700.0,2019-01-09 22:11:00,TX,Austin,78745


### Using `groupby`

In [72]:
# Show the total number of items ordered for each state, city, and zip code.
total_items_groupby = iphone_sales.groupby(['state', 'city', 'zip_code'])['quantity_ordered'].sum()

# Rename the resulting series to reflect the data
total_items_groupby.rename('total_items_ordered', inplace=True)

# Display the top 5 results
top_5_total_items = total_items_groupby.nlargest(5)
top_5_total_items


state  city           zip_code
NY     New York City  10001       40
                      10013       36
       Buffalo        14201       35
TX     Austin         78701       34
CA     Los Angeles    90008       33
Name: total_items_ordered, dtype: int64

### Using `pivot_table`

In [73]:
# Show the total number of items ordered for each city and zip code.
# Create a pivot table for the total number of items ordered
total_items_pivot = iphone_sales.pivot_table(
    values='quantity_ordered',  # Assuming 'quantity_ordered' is the column for the number of items
    index=['city', 'zip_code'],  # Grouping by city and zip code
    aggfunc='sum'  # Summing the quantity ordered
)

# Rename the aggregated column to "Number_Ordered"
total_items_pivot.columns = ['Number_Ordered']

# Sort the pivot table in descending order and display the top 5 results
top_5_total_items = total_items_pivot.sort_values(by='Number_Ordered', ascending=False).head(5)
top_5_total_items


Number_Ordered
city          zip_code                
New York City 10001                 40
              10013                 36
Buffalo       14201                 35
Austin        78701                 34
Los Angeles   90008                 33

### 5. Which day had the most iPhone sales?

In [74]:
# Create a pivot table with the 'order_date' as the index and the "price" as the values.
total_sales_pivot = combined_sales.pivot_table(
    values='price',  # Assuming 'price' represents the sales price
    index='order_date',  # Using 'order_date' as the index
    aggfunc='sum'  # Summing up the prices to get total sales
)

# Rename the aggregated column to "Total_Sales"
total_sales_pivot.columns = ['Total_Sales']

# Display the pivot table
total_sales_pivot


,Total_Sales
order_date,
2019-01-01 03:07:00,11.99
2019-01-01 03:40:00,11.95
2019-01-01 04:56:00,150.00
2019-01-01 05:53:00,2.99
2019-01-01 06:03:00,11.95
...,...
2019-04-01 01:32:00,3.84
2019-04-01 02:22:00,99.99
2019-04-01 02:42:00,389.99


In [75]:
# Resample the pivot table into daily bins 
# and get the total sales for each day rounded to two decimal places.
# Ensure 'order_date' is a datetime index
total_sales_pivot.index = pd.to_datetime(total_sales_pivot.index)

# Resample the pivot table into daily bins and sum the total sales
daily_sales = total_sales_pivot.resample('D').sum()

# Round the results to two decimal places
daily_sales = daily_sales.round(2)

# Sort the resampled pivot table in ascending order based on "Total_Sales"
sorted_daily_sales = daily_sales.sort_values(by='Total_Sales', ascending=True)

sorted_daily_sales

,Total_Sales
order_date,
2019-04-01,5167.43
2019-01-05,46160.62
2019-01-03,46793.74
2019-01-12,47792.25
2019-01-18,48291.98
...,...
2019-03-26,98513.15
2019-03-05,99455.47
2019-03-18,100291.58


### 6.  Which week had the most iPhone sales?

In [76]:
# Resample the pivot table into weekly bins 
# and get the total sales for each week rounded to two decimal places.
# Ensure 'order_date' is a datetime index
total_sales_pivot.index = pd.to_datetime(total_sales_pivot.index)

# Resample the pivot table into weekly bins and sum the total sales
weekly_sales = total_sales_pivot.resample('W').sum()

# Round the results to two decimal places
weekly_sales = weekly_sales.round(2)

# Sort the resampled pivot table in ascending order based on "Total_Sales"
sorted_weekly_sales = weekly_sales.sort_values(by='Total_Sales', ascending=True)

sorted_weekly_sales

,Total_Sales
order_date,
2019-04-07,5167.43
2019-01-06,343062.87
2019-01-20,392745.06
2019-01-13,407539.11
2019-01-27,424518.30
2019-02-03,456563.41
2019-02-24,530962.84
2019-02-10,563327.94
2019-02-17,565026.42
